In [24]:
!pip install -q langchain-google-genai langchain-core langchain pinecone python-dotenv langchain-community langchain-pinecone


In [25]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")


In [26]:
from pinecone import Pinecone
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore  

# Init Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Load and split documents
loader = DirectoryLoader('./data', loader_cls=TextLoader)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = splitter.split_documents(docs)

# Create embeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Upload vectors to Pinecone
vectorstore = PineconeVectorStore.from_documents(  
    documents=splits,
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME  
)

/media/riaz37/WebDevelopment/medical-bot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Error loading file data/Medical_book.pdf


RuntimeError: Error loading data/Medical_book.pdf

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

retriever = Pinecone.from_existing_index(
    index_name=PINECONE_INDEX_NAME,
    embedding=embedding
).as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
query = "What are the symptoms of atrial fibrillation?"
result = qa_chain.invoke(query)
print("Answer:", result["result"])
